# MNIST handwritten digits classification

In [1]:
import keras_tuner as kt
import IPython
import numpy as np
import plotly.express as px
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout


In [2]:
# Load the MNIST handwritten digits dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

img_shape = (*X_train.shape[1:], 1)


In [3]:
def model_builder(hp):
    """
    Build a model with given hyperparameters, which
    are optimized using keras tuner
    """

    # Define hyperparameters
    hp_conv_layers = hp.Int("Convolution layers", min_value=0, max_value=2)
    hp_filters_dict = {
        f"hp_filters_L{i}": hp.Int(f"filters L{i}", min_value=8, max_value=256)
        for i in range(hp_conv_layers)
    }
    hp_padding_dict = {
        f"hp_padding_L{i}": hp.Choice(f"Padding L{i}", values=["same", "valid"])
        for i in range(hp_conv_layers)
    }
    hp_dropout_dict = {
        f"hp_dropout_L{i}": hp.Boolean(f"Dropout L{i}") for i in range(hp_conv_layers)
    }
    hp_dropout_rate_dict = {
        f"hp_dropout_rate_L{i}": hp.Float(f"Dropuot rate L{i}", 0.0, 1.0)
        for i in range(hp_conv_layers)
    }

    hp_dense_layers = hp.Int("Dense Layers", min_value=1, max_value=4)
    hp_nodes_dict = {
        f"hp_nodes_L{i}": hp.Int(f"hp nodes L{i}", min_value=16, max_value=256)
        for i in range(hp_dense_layers)
    }

    # build the model
    model = keras.Sequential([Input(shape=img_shape)])

    for layer_id in range(hp_conv_layers):

        model.add(
            Conv2D(
                filters=hp_filters_dict[f"hp_filters_L{layer_id}"],
                kernel_size=(3, 3),
                activation="relu",
                padding=hp_padding_dict[f"hp_padding_L{layer_id}"],
            )
        )

        model.add(MaxPooling2D(padding="same"))

        if hp_dropout_dict[f"hp_dropout_L{layer_id}"]:
            model.add(Dropout(hp_dropout_rate_dict[f"hp_dropout_rate_L{layer_id}"]))

    model.add(Flatten())

    for layer_id in range(hp_dense_layers):

        model.add(Dense(hp_nodes_dict[f"hp_nodes_L{layer_id}"], activation="relu"))

    model.add(Dense(10, activation="softmax"))

    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss="SparseCategoricalCrossentropy",
        metrics=["accuracy"],
    )

    return model


In [ ]:
tuner = kt.Hyperband(
    model_builder,
    objective="val_accuracy",
    max_epochs=10,
    project_name="handwritten_digits",
)


In [5]:
# defining a call that will clean out output at the end of every training epoch
class ClearTrainingOutput(keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)


stop_early = keras.callbacks.EarlyStopping(patience=3)


In [6]:
tuner.search(
    X_train,
    y_train,
    steps_per_epoch=50,
    validation_split=0.2,
    epochs=10,
    shuffle=True,
    verbose=1,
    callbacks=[stop_early, ClearTrainingOutput()],
)


Trial 30 Complete [00h 00m 19s]
val_accuracy: 0.9797499775886536

Best val_accuracy So Far: 0.9854999780654907
Total elapsed time: 00h 03m 25s
INFO:tensorflow:Oracle triggered exit


In [12]:
# Print details of the best model found
tuner.results_summary(num_trials=1)


Results summary
Results in ./handwritten_digits
Showing 1 best trials
Trial summary
Hyperparameters:
Convolution layers: 2
Dense Layers: 2
hp nodes L0: 232
filters L0: 233
Padding L0: valid
Dropout L0: False
Dropuot rate L0: 0.019850539464430272
hp nodes L1: 177
filters L1: 188
Padding L1: same
Dropout L1: False
Dropuot rate L1: 0.4918100094233566
hp nodes L2: 149
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.9854999780654907


In [11]:
best_model = tuner.get_best_models()[0]
best_model.evaluate(X_test, y_test)


313/313 [==============================] - 1s 2ms/step - loss: 0.0487 - accuracy: 0.9844


[0.048693228513002396, 0.9843999743461609]

In [33]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 233)       2330      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 233)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 188)       394424    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 188)        0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 9212)              0         
                                                                 
 dense (Dense)               (None, 232)               2

In [37]:
import cv2 

img = cv2.imread("bitmap.png", cv2.IMREAD_GRAYSCALE).reshape(1, 28, 28)

print(best_model.predict(img))
np.argmax(best_model.predict(img))

1/1 [==============================] - 0s 9ms/step
[[0.02354246 0.1051085  0.03740197 0.12793149 0.13671263 0.08367053
  0.04042475 0.04816845 0.08856136 0.30847782]]
1/1 [==============================] - 0s 9ms/step


9

In [150]:
# Visualize test images and print the predicted number
import random

img = X_test[random.randint(0, 9999)]

fig = px.imshow(img)

fig.update_layout(
    autosize=False,
    width=300,
    height=300,
    margin=dict(l=10, r=20, t=20, b=20),
)
fig.update_coloraxes(showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)

fig.show()

print(np.argmax(best_model.predict(img.reshape(1, 28, 28))))

1/1 [==============================] - 0s 11ms/step
6
